# Unsupervised Machine Learning Final Project
Aug2023

Data Source: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset

# Project Goals
For this project we'll be looking at text messages and classifying them as spam or legitimate (ham) text messages. The data is labeled, and we'll start with using an unsupervised model and compare it against a supervised model. Stepwise, we'll be doing the following:
1. Import the data
2. Clean the data
3. Explore the data
4. Build the unsupervised model
5. Explore improvements to the unsupervised model
6. Build a supervised model
7. Explore improvements to the supervised model
8. Compare supervised vs. unsupervised models
9. Conclusions

# The Data
This data contains text messages collected from a various number of sources (also described in more detail on the Kaggle source url):
1. 425 spam messages from Grumbletext website
2. 3,375 ham messages from NUS SMS Corpus
3. 450 ham messages from Caroline Tag's PhD thesis
4. 1000 ham messages and 322 spam messages from SMS Spam Corpus v.0.1 Big

In total there are 5572 messages with a breakdown of 87% ham and 13% spam. 

## NOTE
The data doesn't use the default utf-8 encoding, we need to use ISO-8859-1 to read the raw data correctly.

In [5]:
# Import modules
import polars as pl
import altair as alt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from nltk.tokenize import word_tokenize

In [6]:
# Import the data
url = "https://raw.githubusercontent.com/nsxydis/final-project/main/spam.csv"
df = pl.read_csv(url, encoding = 'ISO-8859-1')

In [7]:
# Explore the data


In [ ]:
# Make a class for 
def test(*args):
    print(args)


In [11]:
# Word Embedding
vectorizer = TfidfVectorizer()
embed = vectorizer.fit_transform(df['v2'].to_list())
embed

<5572x8672 sparse matrix of type '<class 'numpy.float64'>'
	with 73916 stored elements in Compressed Sparse Row format>